<div class="licence">
<span>Licence CC BY-NC-ND</span>
<span>Thierry Parmentelat &amp; Arnaud Legout</span>
</div>

# Aller chercher une donnée simple sur Internet

## CSV

Vous trouverez à cette URL un accès aux données de population par pays

<https://population.un.org/wpp/Download/Standard/CSV/>

et plus spécifiquement, le lien pour downloader la donnée

`https://population.un.org/wpp/Download/Files/1_Indicators%20(Standard)/CSV_FILES/WPP2019_TotalPopulationBySex.csv`

Il s'agit pour nous d'écrire

* un code qui va chercher ces données et les traduit en une structure python (donc dans ce cas précis une simple liste de strings)
* avec une autre URL vous pouvez obtenir toutes les populations de toutes les tranches d'âge de la population brésilienne en 2017
* ou encore calculer combien de petits brésiliens seront nés entre aujourd'hui et demain

## Indices

In [ ]:
import json

# il faut installer requests séparément avec
# pip install requests

import requests


In [ ]:
# ici j'ai fait un 'Copy Link Address' depuis chrome, l'espace s'est fait remplacer par un %20
# 20 en hexa = 32 en décimal, c'est le codepoint de Espace
URL = "https://population.un.org/wpp/Download/Files/1_Indicators%20(Standard)/CSV_FILES/WPP2019_TotalPopulationBySex.csv"

# il faut être un peu patient
response = requests.get(URL)


In [ ]:
# dans ce genre de cas dir() est utile
dir(response)

In [ ]:
# mais ça vaut le coup de filter un peu
[symbol for symbol in dir(response) if not symbol.startswith('_')]

In [ ]:

# si tout s'est bien passé ici on doit avoir 200
# c'est un des codes de retours de HTTP
# si c'est par exemple 404 ça signifie que cette URL n'existe plus
response.status_code

In [ ]:
# si on était curieux on pourrait aussi faire
response.headers

In [ ]:
# pour voir par exemple la taille de notre donnée
response.headers['Content-Length']

In [ ]:
# mais bon, bien sûr ce qu'on veut surtout c'est le contenu

type(response.text), response.text[:100]

In [ ]:
# il semble donc qu'on ait affaire à un csv classique
# qu'on peut décortiquer avec object csv.reader
# https://docs.python.org/3/library/csv.html

import csv

# mais pour ça il nous faut un objet de type fichier (file-like object)
# et c'est là que StringIO est super pratique

from io import StringIO

with (StringIO(response.text)) as file_like:
    reader = csv.reader(file_like, delimiter=',')
    # on regarde juste les 3 premiers enregistrements
    for i, row in enumerate(reader, 1):
        print(i, row)
        if i >= 3:
            break


À ce stade, on voit comment on peut transformer nos données en listes Python;
mais on va s'arrêter là, parce qu'en pratique ça c'est vraiment un exercice qu'on ferait en pandas...


## JSON

même exercice mais cette fois on cherche du JSON; je tombe en premier sur ceci

<https://github.com/samayo/country-json/blob/master/src/country-by-population.json>

pour trouver le bon lien, je clique sur 'Raw' et je copie ici

In [ ]:
URL = "https://raw.githubusercontent.com/samayo/country-json/master/src/country-by-population.json"

In [ ]:
def get_url_as_json(url):
    """
    Fetch a URL and decode its result as JSON
    """

    with requests.get(url) as response:
        return json.loads(response.text)

In [ ]:
python_friendly = get_url_as_json(URL)

type(python_friendly)

In [ ]:
python_friendly[:3]

In [ ]:
# et ici par exemple je pourrais décider que c'est plus pratique
# sous la forme d'un dictionnaire name -> population

population = {d['country']: d['population'] for d in python_friendly}


In [ ]:
# de sorte que je peux faire simplement
population['France']